In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ.get('SENDGRID_ALT_KEY')

In [4]:
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))


In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page': 1, 'page_size': 1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
print(response.body)
#print(response.headers)


In [53]:
# Sendgrid retrieve recipients
import os
from sendgrid import SendGridAPIClient


sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.marketing.contacts.exports.post()
id = response.body.decode("utf-8")
id

'{"id":"c7bdc11e-2581-43b2-a218-d18c47543ab0","_metadata":{"self":"https://api.sendgrid.com/v3/marketing/contacts/exports/c7bdc11e-2581-43b2-a218-d18c47543ab0"}}\n'

In [57]:
import json
import time
import pandas as pd
import requests
from io import StringIO

response = sg.client.marketing.contacts.exports._(json.loads(id)['id']).get()
time.sleep(10)
urls = response.body.decode("utf-8")
download = json.loads(urls)["urls"][0]
req = requests.get(download)
df = pd.read_csv(StringIO(req.text))
sg_emails = df['EMAIL'].tolist()

parse index.xml to get latest feed item

In [62]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(newsletter.title)
print(newsletter.description)

ADCN: Team Building, Research Funding & Methods, News
<p>Hope 2026 is off to a great start! This newsletter project is rounding into shape behind the scenes. I’m ready to get your feedback and incorporate any notes you have for ADCN scope and content. My email is at the bottom.</p>
<section class="level3" id="the-science-and-the-art-of-team-building">
<h3 class="anchored">The Science and The Art of Team Building</h3>
<p>Kent Hughes likes his team. He’s the general manager of the Montreal Canadiens, and he told a recent <a href="https://montrealgazette.com/sports/hockey/hockey-inside-out/cowan-young-canadiens-have-each-others-backs-and-it-shows">media gaggle</a> that his team’s recent success was “what happens when a team plays for each other.” As evidence, he pointed to a <a href="https://www.nytimes.com/athletic/6856238/2025/12/03/head-coach-star-player-clash-feedback-nba-nfl/">recent article</a> in <em>The Athletic</em> by Rustin Dodd, that asked “How do you hold a star athlete accou

In [63]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + re.sub(r'"', r'\"', str(s.find('p'))) + '"'
    if s.find('ul'):
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('li').find_next_siblings('li')]) + ']'
    else:
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


{"subject": "ADCN: Team Building, Research Funding & Methods, News", "introduction":[{"p":"<p>Hope 2026 is off to a great start! This newsletter project is rounding into shape behind the scenes. I’m ready to get your feedback and incorporate any notes you have for ADCN scope and content. My email is at the bottom.</p>"}], "sections": [{"section": {"heading": "<h3 class=\"anchored\">The Science and The Art of Team Building</h3>", "description": "<p>Kent Hughes likes his team. He’s the general manager of the Montreal Canadiens, and he told a recent <a href=\"https://montrealgazette.com/sports/hockey/hockey-inside-out/cowan-young-canadiens-have-each-others-backs-and-it-shows\">media gaggle</a> that his team’s recent success was “what happens when a team plays for each other.” As evidence, he pointed to a <a href=\"https://www.nytimes.com/athletic/6856238/2025/12/03/head-coach-star-player-clash-feedback-nba-nfl/\">recent article</a> in <em>The Athletic</em> by Rustin Dodd, that asked “How 

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
emails = [] #..fill as needed
for email in emails:
	sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
	message = Mail(from_email=os.environ.get('SENDGRID_SENDER_EMAIL'), to_emails=email)
	message.dynamic_template_data = eval(handlebar)
	message.template_id = os.environ.get('SENDGRID_TEMPLATE_ID')
	response = sg.send(message)
	print(response.status_code, email)

202 jstenger@uvm.edu


In [ ]:
sg_emails